In [1]:
%cd ..

/Users/peli/Projects/Repositories/DDM-NumEstPark


In [13]:
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import graphviz
from datetime import datetime
from src.model import build_subject_model
from src.utils import get_paths

In [ ]:
def get_sub_paths():
    """
    Return a list of all file paths of csv data
    """
    dir = "data"
    base = Path.cwd()
    data_dir = base / dir
    data_files = list(data_dir.glob('*.csv'))
    print(f"data_files {data_files}")
    return data_files

def get_subject_data():
    files = get_sub_paths()
    file = files[0]
    df = pd.read_csv(file)
    print(df.head(5))
    subject_df = df[df["S_ID"] == "S_00"]

    x_stim = subject_df["Presented_numerosity"]
    y_obs = subject_df["Estimated_numerosity"]

    print(f"Length n_stim {len(x_stim)}")
    print(f"n_stim: {x_stim.value_counts().sort_index()}")
    print(f"Length y_obs {len(y_obs)}")
    print(f"y_obs: {y_obs.value_counts().sort_index()}")
    return x_stim, y_obs

def get_all_data():
    path = get_paths("dataset", "*")

    df = pd.read_csv(path)
    X = df

    return x_stim, y_obs



In [17]:
get_all_data()

data_files [PosixPath('/Users/peli/Projects/Repositories/DDM-NumEstPark/dataset/study2.csv')]
unpacked list


(array([5, 8, 7, ..., 5, 8, 7], shape=(9440,)),
 array([ 5, 10,  8, ...,  6,  8,  8], shape=(9440,)))

In [18]:
n_stim, y_obs = get_all_data()

# convert from pandas series to numpy arrays

print(f"Building model ...")
model = build_subject_model(n_stim, y_obs)


data_files [PosixPath('/Users/peli/Projects/Repositories/DDM-NumEstPark/dataset/study2.csv')]
unpacked list
Building model ...


In [5]:

sample_config = {
    "draws": 2000,
    "tune": 2000,
    "target_accept": 0.9,
    "chains": 4
}

# Sampling
with model:
    result = pm.sample(
        **sample_config
    )

    post_pred = pm.sample_posterior_predictive(
        result,
        var_names=["y_latent", "y_pred"]
    )

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, sigma_0, sigma_1, y_latent]


Output()

Sampling 4 chains for 2_000 tune and 2_000 draw iterations (8_000 + 8_000 draws total) took 7 seconds.
There were 22 divergences after tuning. Increase `target_accept` or reparameterize.
/var/folders/38/7bfzlz1n56s9bgpgqxr75fzh0000gn/T/ipykernel_2897/3674058608.py:14: UserWarning:

The effect of Potentials on other parameters is ignored during posterior predictive sampling. This is likely to lead to invalid or biased predictive samples.

Sampling: [y_latent]


Output()

In [19]:
result


NameError: name 'result' is not defined

In [7]:
post_pred

Inference data with groups:
	> posterior_predictive

In [ ]:
az.plot_trace(result, combined=True)
az.summary(result, round_to=2)

In [ ]:
# posterior predictive: shape (chains, draws, N)
y_latent_samples = post_pred.posterior_predictive.y_latent.values
chains, draws, N = y_latent_samples.shape

# flatten chains + draws
y_flat = y_latent_samples.reshape(chains * draws, N)

# x-axis: stimulus index or actual numerosity
x = np.arange(N)  # or x = n_stim

print(x)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
y_flat = post_pred.posterior_predictive.y_pred.values[0]

In [ ]:
fig = go.Figure()

# Plot posterior latent curves
for sample in y_flat:
    fig.add_trace(
        go.Scatter(
            x=x,
            y=sample,
            mode='lines',
            line=dict(color='lightblue', width=1),
            opacity=0.03,   # very transparent for many curves
            showlegend=False
        )
    )

# Plot observed data
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_obs,
        mode='lines+markers',
        line=dict(color='red', width=3),
        marker=dict(size=6),
        name='Observed'
    )
)

fig.update_layout(
    title='Posterior Predictive Latent Curves vs Observed',
    xaxis_title='Stimulus',
    yaxis_title='Latent response',
    template='plotly_white'
)

fig.show()
